# Ficha de Expressões Regulares 2

### Conceitos mais avançados de expressões regulares

- `.` - corresponde a uma ocorrência de qualquer caracter (exceto '\n', geralmente).
- `\w` - corresponde a um caracter alfanumérico (a-z, A-Z, 0-9 ou _).
- `\W` - corresponde a um caracter **não** alfanumérico.
- `\s` - corresponde a um caracter de *whitespace* (' ', '\t', ou '\n', por exemplo).
- `\S` - corresponde a um caracter que não seja *whitespace*.
- `\d` - corresponde a um dígito.
- `\D` - corresponde a um caracter que não seja um dígito.
- `\btot\w+` - corresponde a uma palavra **começada** por "tot" (o token `\b` representa uma *word boundary*, ou seja, o limite entre um caracter alfanumérico e outro não alfanumérico). Por outras palavras, captura a palavra "totalidade" mas não a palavra "batota". O token `\b` também pode ser usado no fim de palavras.
- `a(?=b)` - corresponde a um caracter `a` que tenha à sua frente um caracter `b`, mas não captura o caracter `b`. (*positive lookahead*)
- `a(?!b)` - corresponde a um caracter `a` que **não** tenha à sua frente um caracter `b`, mas não captura o caracter seguinte. (*negative lookahead*)
- `a(?<=b)` - corresponde a um caracter `a` que tenha atrás de si um caracter `b`, mas não captura o caracter `b`. (*positive lookbehind*)
- `a(?<!b)` - corresponde a um caracter `a` que **não** tenha atrás de si um caracter `b`, mas não captura o caracter anterior. (*negative lookbehind*)


Podemos usar *grupos de captura* em expressões regulares para isolar segmentos da string capturada. Usamos parênteses para definir grupos de captura.

In [ ]:
import re
m = re.search(r'(2[0-3]|[0-1][0-9]):([0-5][0-9])', "13:49")

print(m.groups()) # conjunto dos grupos de captura
print(m.group(0)) # toda a string capturada
print(m.group(1)) # o primeiro grupo de captura

('13', '49')
13:49
13


O módulo re possui ainda *flags* que podemos usar nas suas funções. As mais úteis são:

- `re.I` ou `re.IGNORECASE`: faz uma correspondência *case insensitive*.
- `re.M` ou `re.MULTILINE`: os tokens de âncora `^` e `$` passam a corresponder ao início/fim de cada linha, em vez do início/fim de uma string.
- `re.S` ou `re.DOTALL`: o token `.` passa a corresponder também a um caracter `\n`.

Podemos usar estas flags da seguinte forma: `re.search(r'trans.*mar', "TRANSF\nORMAR", re.I | re.S)`

## Exercício 1 - Conversão de datas

Define a função `iso_8601` que converte as datas presentes numa string no formato DD/MM/AAAA para o formato ISO 8601 - AAAA-MM-DD, usando expressões regulares e grupos de captura.

In [12]:
import re
texto = """A 03/01/2022, Pedro viajou para a praia com a sua família.
Eles ficaram hospedados num hotel e aproveitaram o sol e o mar durante toda a semana.
Mais tarde, no dia 12/01/2022, Pedro voltou para casa e começou a trabalhar num novo projeto.
Ele passou muitas horas no escritório, mas finalmente terminou o projeto a 15/01/2022."""
def iso_8601(string):
    list = re.findall('\d{2}/\d{2}/\d{4}',string)
    for i in list:
        res = print(i[-4:]+'-'+i[3:5]+'-'+i[0:2])
    return res

iso_8601(texto)


2022-01-03
2022-01-12
2022-01-15


## Exercício 2 - Validação de ficheiros

Escreve um programa que lê uma lista de nomes de ficheiros e determina se cada nome é válido ou não. O nome de um ficheiro deve conter apenas caracteres alfanuméricos, hífens, underscores ou pontos, seguido de uma extensão (e.g., ".txt", ".png", etc.).

In [51]:
file_names = [
  "document.txt", # válido
  "file name.docx", # inválido
  "image_001.jpg", # válido
  "script.sh.txt", # válido
  "test_file.txt", # válido
  "file_name.", # inválido
  "my_resume.docx", # válido
  ".hidden-file.txt", # válido
  "important-file.text file", # inválido
  "file%name.jpg" # inválido
]

for i in file_names:
  if (bool(re.match(r'^(\w|\.|_|-)*(\.[a-z]+)+$',i))):
    print('File name valid: "',i,'"')
  else: print('File name invalid. "',i, '"')

File name valid: " document.txt "
File name invalid. " file name.docx "
File name valid: " image_001.jpg "
File name valid: " script.sh.txt "
File name valid: " test_file.txt "
File name invalid. " file_name. "
File name valid: " my_resume.docx "
File name valid: " .hidden-file.txt "
File name invalid. " important-file.text file "
File name invalid. " file%name.jpg "


### Alínea 2.1

Modifica o programa anterior para colocar os nomes de ficheiro válidos num dicionário, no qual as chaves deverão ser as extensões dos mesmos. Por outras palavras, agrupa os ficheiros por extensão.

In [79]:
dic = {}

for i in file_names:
    if (bool(re.match(r'^(\w|\.|_|-)*(\.[a-z]+)+$',i))):
     
        ending = re.findall('\.[a-z]+$',i)[0]
       
        if ending in dic:
            dic[ending].append(i)
        else: 
            dic[ending] = [i]

dic

{'.txt': ['document.txt',
  'script.sh.txt',
  'test_file.txt',
  '.hidden-file.txt'],
 '.jpg': ['image_001.jpg'],
 '.docx': ['my_resume.docx']}

## Exercício 3 - Conversão de nomes

Escreve um filtro de texto que converte cada nome completo de uma pessoa encontrada num texto fonte, no formato `PrimeiroNome SegundoNome [...] UltimoNome` para o formato `UltimoNome, PrimeiroNome`. Por exemplo, "Rui Vieira de Castro" passa a "Castro, Rui". Atenção aos conectores "de", "dos", etc.

In [105]:
texto = """Este texto foi feito por Sofia Guilherme Rodrigues dos Santos, com 
base no texto original de Pedro Rafael Paiva Moura, com a ajuda
dos professores Pedro Rangel Henriques e José João Antunes Guimarães Dias De Almeida.
Apesar de partilharem o mesmo apelido, a Sofia não é da mesma família do famoso
autor José Rodrigues dos Santos."""

import unicodedata

texto = unicodedata.normalize('NFKD', texto)
texto = texto.encode('ASCII', 'ignore').decode('utf-8')
texto = texto.replace('\n',' ')
texto

'Este texto foi feito por Sofia Guilherme Rodrigues dos Santos, com  base no texto original de Pedro Rafael Paiva Moura, com a ajuda dos professores Pedro Rangel Henriques e Jose Joao Antunes Guimaraes Dias De Almeida. Apesar de partilharem o mesmo apelido, a Sofia nao e da mesma familia do famoso autor Jose Rodrigues dos Santos.'

In [136]:
list = re.split('\W',texto)
print(list)


['Este', 'texto', 'foi', 'feito', 'por', 'Sofia', 'Guilherme', 'Rodrigues', 'dos', 'Santos', '', 'com', '', 'base', 'no', 'texto', 'original', 'de', 'Pedro', 'Rafael', 'Paiva', 'Moura', '', 'com', 'a', 'ajuda', 'dos', 'professores', 'Pedro', 'Rangel', 'Henriques', 'e', 'Jose', 'Joao', 'Antunes', 'Guimaraes', 'Dias', 'De', 'Almeida', '', 'Apesar', 'de', 'partilharem', 'o', 'mesmo', 'apelido', '', 'a', 'Sofia', 'nao', 'e', 'da', 'mesma', 'familia', 'do', 'famoso', 'autor', 'Jose', 'Rodrigues', 'dos', 'Santos', '']


## Exercício 4 - Códigos postais 2

Define uma função `codigos_postais` que recebe uma lista de códigos postais e divide-os com base no hífen. Ao contrário do exercício da ficha anterior, esta função pode receber códigos postais inválidos. A função deve devolver uma lista de pares e apenas processar cada linha uma vez.

In [138]:
lista = [
    "4700-000", # válido
    "9876543", # inválido
    "1234-567", # válido
    "8x41-5a3", # inválido
    "84234-12", # inválido
    "4583--321", # inválido
    "9481-025" # válido
]

for i in lista:
    print(re.match('\d{4}-\d{3}',i))

<re.Match object; span=(0, 8), match='4700-000'>
None
<re.Match object; span=(0, 8), match='1234-567'>
None
None
None
<re.Match object; span=(0, 8), match='9481-025'>


## Exercício 5 - Expansão de abreviaturas

Escreve um filtro de texto que expanda as abreviaturas que encontrar no texto fonte no formato "/abrev".

In [168]:
abreviaturas = {
    "UM": "Universidade do Minho",
    "LEI": "Licenciatura em Engenharia Informática",
    "UC": "Unidade Curricular",
    "PL": "Processamento de Linguagens"
}

texto = "A /abrev{UC} de /abrev{PL} é muito fixe! É uma /abrev{UC} que acrescenta muito ao curso de /abrev{LEI} da /abrev{UM}."

list = re.split('\s',texto)

newlist = []
for i in list:
    if bool(re.match('/abrev{[A-Z]+}',i)):
        newlist.append(abreviaturas[re.findall('[A-Z]+',i)[0]]) 
    else:
        newlist.append(i)

' '.join(newlist)

'A Unidade Curricular de Processamento de Linguagens é muito fixe! É uma Unidade Curricular que acrescenta muito ao curso de Licenciatura em Engenharia Informática da Universidade do Minho'

## Exercício 6 - Matrículas

Define uma função `matricula_valida` que recebe uma string de texto e determina se esta contém uma matrícula válida. Uma matrícula segue o formato AA-BB-CC, no qual dois dos três conjuntos devem ser compostos por números e o terceiro por letras maiúsculas (por exemplo, 01-AB-23), ou o novo formato no qual dois dos conjuntos são compostos por letras maiúsculas e o terceiro por números (por exemplo, 89-WX-YZ). Os conjuntos podem ser separados por um hífen ou um espaço.

Extra: Garante que o mesmo separador é usado para separar os três conjuntos.

In [170]:
matriculas = [
    "AA-AA-AA", # inválida
    "LR-RB-32", # válida
    "1234LX", # inválida
    "PL 22 23", # válida
    "ZZ-99-ZZ", # válida
    "54-tb-34", # inválida
    "12 34 56", # inválida
    "42-HA BQ" # válida, mas inválida com o requisito extra
]

for i in matriculas:
    list = re.split('-|\W',i)
    c_letter_blocks = 0
    c_digit_blocks = 0
    for j in list:
        if re.match('[A-Z]{2}',j):
            c_letter_blocks +=1
        elif re.match('\d{2}',j):
            c_digit_blocks += 1
    if (c_digit_blocks == 2 and c_letter_blocks == 1) or (c_digit_blocks == 1 and c_letter_blocks == 2):
        print(i, ' is a valid number plate')
    else: print(i, ' is not valid')


AA-AA-AA  is not valid
LR-RB-32  is a valid number plate
1234LX  is not valid
PL 22 23  is a valid number plate
ZZ-99-ZZ  is a valid number plate
54-tb-34  is not valid
12 34 56  is not valid
42-HA BQ  is a valid number plate


## Exercício 7 - *Mad Libs*

O jogo *Mad Libs*, bastante comum em países como os Estados Unidos, consiste em pegar num texto com espaços para algumas palavras e preencher esses espaços de acordo com o tipo de palavra que é pedida.

Escreve um programa que lê um texto no formato *Mad Libs* e pede ao utilizador para fornecer palavras que completem corretamente o texto.

In [186]:
texto = """Num lindo dia de [ESTAÇÃO DO ANO], [NOME DE PESSOA] foi passear com o seu [EXPRESSÃO DE PARENTESCO MASCULINA]. 
Quando chegaram à [NOME DE LOCAL FEMININO], encontraram um [OBJETO MASCULINO] muito [ADJETIVO MASCULINO].
Ficaram muito confusos, pois não conseguiam identificar a função daquilo.
Seria para [VERBO INFINITIVO]? Tentaram perguntar a [NOME DE PESSOA FAMOSA], que também não sabia.
Desanimados, pegaram no objeto e deixaram-no no [NOME DE LOCAL MASCULINO] mais próximo. 
Talvez os [NOME PLURAL MASCULINO] de lá conseguissem encontrar alguma utilidade para aquilo."""

list = re.split('\W(?=\[)|(?<=\])\W',texto)
cleanlist = []
for i in list:
    if i == '':
        pass
    else:
        cleanlist.append(i.strip())

for i in cleanlist:
    if re.match('\[',i):
        request = 'Give a '+i
        a = input(request)
        print(a)
    else: print(i)


Num lindo dia de
verao
florian
foi passear com o seu
padre
Quando chegaram à
anna
encontraram um
seu
muito
lindo
Ficaram muito confusos, pois não conseguiam identificar a função daquilo.
Seria para

Tentaram perguntar a

que também não sabia.
Desanimados, pegaram no objeto e deixaram-no no

mais próximo. 
Talvez os

de lá conseguissem encontrar alguma utilidade para aquilo.


## Exercício 8 - Remoção de repetidos

Escreve um filtro de texto que sempre que encontrar no texto fonte uma palavra repetida elimina as repetições, ou seja, substitui a lista de palavras por 1 só palavra.

In [199]:
# Generate text with dublicates

from random import randint


text ='Ich liebe es, wenn die Sonne scheint und alles so schön bunt und lebendig ist. Die Menschen sind fröhlich und voller Energie und ich fühle mich wirklich glücklich und zufrieden. Es ist wie eine Oase in der Wüste des Alltags, die mir neue Kraft gibt und mich motiviert, weiterzumachen'

list = re.split('\s',text)
newlist = []
for i in list:
    if randint(1,10)<= randint(1,10):
        newlist.append(i)
        newlist.append(i)
    else: newlist.append(i)

newtext = ' '.join(newlist)
print(newtext)

Ich Ich liebe liebe es, wenn wenn die Sonne scheint und und alles alles so so schön schön bunt bunt und und lebendig lebendig ist. Die Die Menschen sind sind fröhlich und und voller Energie Energie und ich ich fühle mich wirklich wirklich glücklich und zufrieden. Es Es ist wie wie eine Oase Oase in der der Wüste des Alltags, die die mir mir neue neue Kraft Kraft gibt gibt und und mich mich motiviert, weiterzumachen weiterzumachen


In [205]:
# Clean text with dublicates

list = re.split('\s',newtext)
newlist=[]
for i in range(len(list)-1):
    if list[i] == list[i+1]:
        pass
    else:
        newlist.append(list[i])

newtext = ' '.join(newlist)
print(newtext)

Ich liebe es, wenn die Sonne scheint und alles so schön bunt und lebendig ist. Die Menschen sind fröhlich und voller Energie und ich fühle mich wirklich glücklich und zufrieden. Es ist wie eine Oase in der Wüste des Alltags, die mir neue Kraft gibt und mich motiviert,
